# Gmarket

In [22]:
from bs4 import BeautifulSoup

with open('gmarket_example2.html', 'r', encoding='utf-8') as file:
    html_content = file.read()


soup = BeautifulSoup(html_content, 'html.parser')
options = soup.find_all('span', class_='item_tit')
li = []
for title in options:
    li.append(title.text.strip())

li = set(li) #delete distinct

for option in li:
    print(option)

농심 컵라면 세트 육개장 사발면 육계장 컵라면 + 신라면 큰사발 + 짜장라면 짜파게티 큰컵라면 18개
농심 컵라면 세트 큰사발 육개장 사발면 5개+신라면 큰컵라면 3개+튀김우동 3개+너구리 큰컵 3개  14개
농심 컵라면 세트 큰사발 육개장 사발면 5개+신라면 큰컵 5개+ 짜파게티 큰컵4개 다양한 간식 야식 14개
농심 컵라면 세트 큰사발 육개장 사발면 신라면 튀김우동 짜파게티 짜장 너구리 큰 컵 간식 야식  14개
농심 컵라면 세트 큰사발 육개장 사발면 5개+신라면 큰컵 5개+ 튀김우동 큰컵4개 간식 야식 14개
농심 컵라면 세트 육개장 사발면 육계장+김치사발면+신라면 큰사발+튀김우동 큰컵라면 간식 야식 18개
농심 컵라면 세트 육개장 사발면 육계장+김치사발면 컵라면 공장 회사 학원 사무실 간식 야식 24개
농심 컵라면 세트 육개장 사발면+김치사 발면+신라면 큰사발+짜파게티 큰컵라면 간식 야식 다양한 18개
농심 컵라면 세트 큰컵 라면 큰사발 육개장 사발면 5개+신라면 3개+너구리 3개+오징어짬뽕 3개 간식 14개
농심 컵라면 세트 큰사발 육개장 사발면 5개+신라면 큰컵라면 3개+새우탕면 큰컵 3개+김치 큰컵 3개  14개
